<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/leogabac-Lab2/leogabac_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix Factorization**
**Leonardo Gabriel Alanis Cantú**

# **Abstract**

Matrices are a mathematical object that come to be very useful in many areas of Science. In this report we will explore some topics related to matrices, such as CSR sparse matrices, QR factorization, computation of the inverse to solve systems of linear equations, as well as an approximation method when a system of equations has no solution. The algorithms are explained in the Introduction section, and implemented in Python code in the Methods section. Results were fairly accurate in the different tests that were performed.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [ ]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
from sympy import *
import math

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

Describe the methods you used to solve the problem. This may be a combination of text, mathematical formulas (Latex), algorithms (code), data and output.  

Sparse matrices, are matrices for which most of its elements are zeroes. Since anything multiplied by zero is zero, the natural question arises: how can we take advantage of the structure of these matrices to store them in a more compacted way? The _Compressed Sparse Row_ format, stores the values of a matrix in three different lists

- ´vals´ contains the non-zero values of $A$.
- ´col_idx´ contains the column index of non-zero entries.
- ´row_ptr´ contains the non-zero index for which a new row starts.

Matrix-vector multiplication is implemented as always, but with a few subtleties regarding the CSR format.

A $m\times n$ matrix can be factorized in the product of two matrices $A = QR$ where $Q$ is orthogonal i.e. $Q Q^{\text{T}} = I$, and $R$ is an upper triangular matrix. As for the scope of this report, we will consider $A$ a full rank matrix, for which we will form the columms of $Q$ by the Gram-Schmidth process with the columns of $A$. (The details will be taken from the textbook by Hoffman, J.)

Given a system of linear equations, it can always be boiled down to the matrix equation $Ax = b$, for which $x$ is unknown. A solver will be implemented in this report by taking advantage that $A$ has a unique solution if it is invertible (which is what we want), or well _full rank_. We reuse the QR factorization algorithm that was implemented, that is

$$
A^{-1} = \left(QR\right)^{-1} = R^{-1}Q^{-1} = R^{-1} Q^{\text{T}}
$$

This means that, in order to solve $Ax = b$, we need to multiply $x = R^{-1}Q^{\text{T}}b$. 

The matrix $R^{-1}$ seems to be repeating the same problem, but it is not, we can take the advantage that it is by definition an upper triangular, and find its inverse by gaussian reducing it into its unique reduced echelon form.

Finally, suppose that $Ax = b$ is an inconsistent system, meaning that there is no solution; the problem arises: which is _the closest_ solution? This is known as a _least squares_ solution. The closest approximation is given by the solution to the new system of equations

$$
A^{\text{T}}A x = A^{\text{T}} b
$$

However, there is a problem, it is possible that $A^{\text{T}}A$ is not invertible, meaning that there is an infinite set of solutions, we will not deal with such cases in this report. Since we want $A^{\text{T}}A$ to be invertible, we want the columns of $A$ to be linearly independent, and that can only happen if $m > n$, hence those are the type of cases that will be taken into consideration.

All four algorithms will be implemented in the following section of the report.


# **Method**

CRS Matrix-vector product

In [ ]:
class sparse_crs:
    # Object that contains CSR format information
    def __init__(self, vals, colind, rowptr):
        self.vals = vals
        self.colind = colind
        self.rowptr = rowptr

# Usually, I wouldn't need this function, but it is for testing purposes
def mat2crs(A):
    # Input
    # A: sparse matrix
    
    # Output
    # CSR data val, col_ind, row_ptr
    vals = []
    colind = []
    rowptr = []
    nzind = 0
    
    for row in range(A.shape[0]):
        rowptr.append(nzind)
        for col in range(A.shape[1]):
            if A[row,col] != 0:
                nzind += 1
                vals.append( A[row,col] )
                colind.append(col)
    rowptr.append( rowptr[-1]+1)
    
    return [vals, colind, rowptr]


def crs_matrixvec(A,x):
    # Input
    # A: sparse CSR object
    # x: vector
    
    # Output
    # y = Ax
    
    n = len(x)
    y = np.zeros(n) # initialize
    
    for i in range(n):
        for j in range( A.rowptr[i],A.rowptr[i+1]):
            y[i] += A.vals[j]*x[ A.colind[j] ]
    
    return y


QR factorization

In [ ]:
def QR_factorization_GS(A):
    # Input
    # A: n x n invertible matrix
    
    # Output
    # Q: Orthogonal n x n matrix
    # R: n x n upper triangular matrix
    
    n = A.shape[0]
    Q = np.zeros(A.shape) # initialize
    R = np.zeros(A.shape) # initialize
    
    for j in range(n):
        v = A[:,j]
        for i in range(j):
            R[i,j] = np.dot(Q[:,i],v)
            v = v - R[i,j]*Q[:,i]
        
        R[j,j] = math.sqrt( np.dot(v,v) )
        Q[:,j] = v / R[j,j]
        
    return Q, R

Solver of a system of linear equations

In [ ]:
def reduceR(R):
    # Input
    # R: n x n upper triangular invertible matrix
    
    # Output
    # Rinv: inverse matrix of R
    
    n = R.shape[0]
    R = np.concatenate( (R, np.identity(n)) , axis = 1)
    for col in reversed(range(n)):
        toone = np.identity(n) # initialize
        tozero = np.identity(n) # initialize
        
        toone[col,col] = 1/R[col,col]
        tozero[0:col,col] = - R[0:col,col]
        E = np.dot(tozero,toone)
        R = np.dot(E,R)
    return R[:,n:2*n]

def solvelinear(A,b):
    # Input
    # A: n x n invertible matrix
    # b: n-vector
    
    # Output
    # x: solution to Ax = b
    
    Q,R = QR_factorization_GS(A)
    Rinv = reduceR(R)
    Ainv = np.dot(Rinv,np.transpose(Q))
    
    return np.dot(Ainv,b)
    

Least squares solution

In [ ]:
def leastsolve(A,b):
    # Input
    # A: m x n matrix, s.t. m>n
    # b: m element vector
    
    # Output
    # x: best approximation of a solution
    
    newA = np.dot(np.transpose(A),A)
    newb = np.dot(np.transpose(A),b)
    pivots = Matrix(newA).rref()[1]
    assert len(pivots) == newA.shape[1], "Infinite number of solutions"
    
    x = solvelinear(newA,newb) # the one that we programmed earlier
    return x

# **Results**

For the CSR matrix-vector product, we will test on an arbitrary sparse matrix, and compare it with the usual matrix-vector product from the numpy library.

In [ ]:
A = np.array([ [2,0,0,2,0], [3,4,2,5,0], [5,0,0,8,17], [0,0,10,16,0], [0,0,0,0,14] ]) # declare array
sparseA = sparse_crs(*mat2crs(A)) # convert to csr
x = np.array([1,2,3,4,5]) # 

start_time = time.time()
y1 = crs_matrixvec(sparseA,x)
print("CSR: %s seconds" % (time.time() - start_time))

start_time = time.time()
y2 = np.dot(A,x)
print("Dense: %s seconds" % (time.time() - start_time))

print("|y2-y1| =", np.linalg.norm(y2-y1))

CSR: 0.0 seconds
Dense: 0.0009970664978027344 seconds
|y2-y1| = 0.0


For the QR factorization we will use the Frobenius norm
$$
\| A \|_F = \sqrt{ \text{tr} (A A^{\dagger} ) }
$$
where $A^\dagger$ is the adjoint matrix, or the transpose for real matrices. Hence we will compute
$$
\| Q Q^{\text{T}} - I \|_F,
$$
and
$$
\| Q R - A \|_F.
$$

In [ ]:
def frobenius(A):
    return math.sqrt(np.trace(  np.dot( A, np.transpose(A) ) ))

A = np.array([ [2,-1],[-1,2] ])
Q,R = QR_factorization_GS(A)

print("|QQ^T - I| =", frobenius( np.dot(Q, np.transpose(Q)) - np.identity(2)  ))
print("|QR - A| =", frobenius( np.dot(Q,R) - A ))

|QQ^T - I| = 3.1836122848239643e-16
|QR - A| = 0.0


To test the solver of linear equations, we will compute the norms $
\| Ax - b\|$, and $\| x-y\|$ where $y$ is the real solution. 

In [ ]:
A = np.array([ [1,-2,1],[0,2,-8],[-4,5,9] ])
b = np.array([0,8,-9])
x = solvelinear(A,b) 
y = np.array([29,16,3])

print("|Ax - b| =", np.linalg.norm(np.dot(A,x) - b ) )
print("|x-y| =", np.linalg.norm(x-y) )

|Ax - b| = 1.724530548492318e-13
|x-y| = 6.316845058634323e-12


For the least square solution, we will take a matrix where $m>n$ (as explained in the Introduction), and compute the residual
$$
\|Ax - b\|
$$

In [ ]:
A = np.array([[0, 2,4], [4, 1,2], [-2, 3,5], [-3,2,7]])
b = np.array([ [4],[5],[6],[4] ]);
x = leastsolve(A,b)
print("|Ax - b| =", np.linalg.norm(np.dot(A,x) - b ) )

|Ax - b| = 0.87912444697435


# **Discussion**

In this report we explored some algorithms for matrix factorization, which were later useful to solve systems of linear equations, as well as sparse matrix-vector multiplication. 

In general, the algorithms performed pretty well, even the Gram-Schmidt QR decomposition which should be one of the slowest, and the most "inaccurate" from floating point operation accumulation. However, from the tests that were done (which were not at a big scale), the algoritms did surprisingly well.

There are a few things that need to be explored out of this report, that could be important implementations to some simulations. The first one is the implementation of sparse matrix-matrix multiplication, since this could help improve the simulation of time evolution in 1D Quantum systems. In one dimension, with a time independent Hamiltonian $\hat{H}$, the evolution of a quantum state $\psi(x,0)$ is governed by the unitary propagator

$$
\psi(x,t) = e^{-i\hat{H}t/\hbar}\psi(x,0)
$$

Where $\hat{H}$ is the Hamiltonian, which from the correspondance principle can be written as

$$
-\dfrac{\hbar^2}{2m}\dfrac{\text{d}}{\text{d}x^2} + V(x),
$$

In general, this is an infinite-dimensional matrix, but can be approximated with the differential matrix operator seen in the lectures, but with a really big matrix. The problem with this time-evolution, is that the matrix-exponential is an infinite-series that needs to multiply $\hat{H}$ an incredibly amout of times, which is computationally very intensive. 

In matrix representation $\hat{H}$ is usually a sparse matrix, hence I believe that this algorithm could be more efficient if we implement matrix exponentiation with sparse matrix-matrix multiplication.

Another thing that can be improved in the implementations given in this report, is the least squares solution whenever there is an infinite amount of solutions, these cases were not implemented since I do not know how to deal with them